In [2]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")
Dataset_name = "NCI1"

# *BFS Positional Encodings with Variable k-Roots in GNNs*

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"
K_VALUES = [2,3,5,8,10]

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8, k=3):
        super().__init__()
        self.bfs_proj = torch.nn.Sequential(
            torch.nn.Linear(k, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, bfs_ranks, edge_index, batch):
        bfs_feat = self.bfs_proj(bfs_ranks)
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_transform(k):
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_ranks = torch.full((data.num_nodes, k), -1.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            degrees = dict(G.degree())
            top_k_nodes = sorted(degrees, key=degrees.get, reverse=True)[:k]
            for idx, root in enumerate(top_k_nodes):
                bfs_order = list(nx.bfs_tree(G, root))
                L = max(1, len(bfs_order))
                for i, node in enumerate(bfs_order):
                    bfs_ranks[node, idx] = i / L
        data.bfs = bfs_ranks
        if data.x is None:
            data.x = torch.ones((data.num_nodes, 1))  # default feature
        return data
    return transform

def run_one_k(k_roots):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for SEED in range(1, NUM_SEEDS + 1):
        set_seed(SEED)

        # Load dataset safely
        try:
            original_dataset = TUDataset(
                root='data/TUDataset',
                name=Dataset_name,
                transform=make_bfs_transform(k_roots)
            )
        except Exception as e:
            print(f"Error loading dataset: {e}")
            return

        # Filter None entries but keep original_dataset for num_classes
        dataset = [d for d in original_dataset if d is not None]
        if len(dataset) == 0:
            print("No valid graphs in dataset")
            return

        # Shuffle dataset
        random.shuffle(dataset)

        # Ensure features exist
        if dataset[0].x is None:
            for data in dataset:
                data.x = torch.ones((data.num_nodes, 1))

        # Get num_classes from the original dataset
        num_classes = original_dataset.num_classes
        if num_classes is None:
            num_classes = int(max([d.y.max().item() for d in dataset]) + 1)

        num_graphs = len(dataset)
        train_size = int(0.8 * num_graphs)
        train_dataset = dataset[:train_size]
        test_dataset = dataset[train_size:]

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

        wandb.init(
            project=PROJECT,
            entity=ENTITY,
            name=f"GCN_BFS_MaxDeg_{k_roots}Roots_seed_{SEED}",
            group=f"GCN_{Dataset_name}_BFS_K_Roots",
            job_type="bfs-k-roots",
            reinit=True,
            config={
                "model": "GCN",
                "hidden_channels": 64,
                "epochs": EPOCHS,
                "lr": 0.001,
                "dataset": Dataset_name,
                "task": f"BFS_{k_roots}_Roots",
                "bfs_k_roots": k_roots,
                "bfs_root_criterion": "max_degree",
                "seed": SEED,
                "cuda_available": torch.cuda.is_available(),
                "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
                "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
            }
        )

        model = GCN(
            in_channels=dataset[0].x.shape[1],
            hidden_channels=64,
            out_channels=num_classes,
            k=k_roots
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        def train():
            model.train()
            total_loss = 0.0
            for data in train_loader:
                data = data.to(device)
                out = model(data.x, data.bfs, data.edge_index, data.batch)
                loss = F.nll_loss(out, data.y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item() * data.num_graphs
            return total_loss / len(train_dataset)

        def test():
            model.eval()
            correct = 0
            for data in test_loader:
                data = data.to(device)
                out = model(data.x, data.bfs, data.edge_index, data.batch)
                pred = out.argmax(dim=1)
                correct += int((pred == data.y).sum())
            return correct / len(test_dataset)

        for epoch in range(1, EPOCHS + 1):
            loss = train()
            test_acc = test()
            # compute train accuracy
            model.eval()
            correct_train = 0
            for data in train_loader:
                data = data.to(device)
                out = model(data.x, data.bfs, data.edge_index, data.batch)
                pred = out.argmax(dim=1)
                correct_train += int((pred == data.y).sum())
            train_acc = correct_train / len(train_dataset)
            print(f"K={k_roots} | Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")
            wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

        wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for k in K_VALUES:
        run_one_k(k)


Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


K=2 | Seed 1 | Epoch 001 | Loss: 0.6874 | Test Acc: 0.5061
K=2 | Seed 1 | Epoch 002 | Loss: 0.6704 | Test Acc: 0.6119
K=2 | Seed 1 | Epoch 003 | Loss: 0.6548 | Test Acc: 0.6058
K=2 | Seed 1 | Epoch 004 | Loss: 0.6403 | Test Acc: 0.6168
K=2 | Seed 1 | Epoch 005 | Loss: 0.6328 | Test Acc: 0.5912
K=2 | Seed 1 | Epoch 006 | Loss: 0.6243 | Test Acc: 0.6411
K=2 | Seed 1 | Epoch 007 | Loss: 0.6151 | Test Acc: 0.6472
K=2 | Seed 1 | Epoch 008 | Loss: 0.6048 | Test Acc: 0.6460
K=2 | Seed 1 | Epoch 009 | Loss: 0.6057 | Test Acc: 0.6496
K=2 | Seed 1 | Epoch 010 | Loss: 0.5942 | Test Acc: 0.6521
K=2 | Seed 1 | Epoch 011 | Loss: 0.5930 | Test Acc: 0.6521
K=2 | Seed 1 | Epoch 012 | Loss: 0.5895 | Test Acc: 0.6338
K=2 | Seed 1 | Epoch 013 | Loss: 0.5871 | Test Acc: 0.6241
K=2 | Seed 1 | Epoch 014 | Loss: 0.5872 | Test Acc: 0.6521
K=2 | Seed 1 | Epoch 015 | Loss: 0.5802 | Test Acc: 0.6691
K=2 | Seed 1 | Epoch 016 | Loss: 0.5791 | Test Acc: 0.6740
K=2 | Seed 1 | Epoch 017 | Loss: 0.5829 | Test Acc: 0.65

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▆▆▅▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁
test_acc,▁▅▅▄▆▇▇▇▇▆▇█▇▇▆▇█▇▇▆█▆▇▇█▇▇██▆█▇█████▇██
train_acc,▁▄▃▆▆▆▆▆▆▅▇▇▇▇▆▇▇▇▇▆▇▆▇▇▇▇▇██▆███████▇██
epoch,50
loss,0.54788
test_acc,0.67518
train_acc,0.73266


K=2 | Seed 2 | Epoch 001 | Loss: 0.6872 | Test Acc: 0.5280
K=2 | Seed 2 | Epoch 002 | Loss: 0.6756 | Test Acc: 0.6022
K=2 | Seed 2 | Epoch 003 | Loss: 0.6642 | Test Acc: 0.6204
K=2 | Seed 2 | Epoch 004 | Loss: 0.6545 | Test Acc: 0.6436
K=2 | Seed 2 | Epoch 005 | Loss: 0.6402 | Test Acc: 0.6448
K=2 | Seed 2 | Epoch 006 | Loss: 0.6325 | Test Acc: 0.6484
K=2 | Seed 2 | Epoch 007 | Loss: 0.6236 | Test Acc: 0.6204
K=2 | Seed 2 | Epoch 008 | Loss: 0.6211 | Test Acc: 0.6484
K=2 | Seed 2 | Epoch 009 | Loss: 0.6196 | Test Acc: 0.6509
K=2 | Seed 2 | Epoch 010 | Loss: 0.6068 | Test Acc: 0.6606
K=2 | Seed 2 | Epoch 011 | Loss: 0.6066 | Test Acc: 0.6509
K=2 | Seed 2 | Epoch 012 | Loss: 0.6051 | Test Acc: 0.6727
K=2 | Seed 2 | Epoch 013 | Loss: 0.6002 | Test Acc: 0.6813
K=2 | Seed 2 | Epoch 014 | Loss: 0.5962 | Test Acc: 0.6788
K=2 | Seed 2 | Epoch 015 | Loss: 0.5913 | Test Acc: 0.6776
K=2 | Seed 2 | Epoch 016 | Loss: 0.5884 | Test Acc: 0.6752
K=2 | Seed 2 | Epoch 017 | Loss: 0.5961 | Test Acc: 0.67

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▇▆▆▅▅▅▄▄▄▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▂
test_acc,▁▄▄▅▅▄▅▅▆▅▇▆▆▆▆▇▇▇▇▇▇▆██▆▇█▇▇▇▆█▆██▇▇▆█▇
train_acc,▁▃▄▅▅▅▅▆▆▆▆▇▆▆▆▇▇▆▇▇██▆████▇█▇▇█▆██▇█▇█▇
epoch,50
loss,0.56011
test_acc,0.6837
train_acc,0.7062


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
test_acc,▁▅▅▅▄▄▄▆▅▆▇▆▆▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇██▇▇▇▆▇▆█▇▇█
train_acc,▁▄▄▅▃▄▄▆▅▅▆▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▆█▇█▇█▇█
epoch,50
loss,0.50978
test_acc,0.72384
train_acc,0.7646


K=8 | Seed 28 | Epoch 001 | Loss: 0.6899 | Test Acc: 0.6229
K=8 | Seed 28 | Epoch 002 | Loss: 0.6766 | Test Acc: 0.5353
K=8 | Seed 28 | Epoch 003 | Loss: 0.6654 | Test Acc: 0.5985
K=8 | Seed 28 | Epoch 004 | Loss: 0.6536 | Test Acc: 0.6521
K=8 | Seed 28 | Epoch 005 | Loss: 0.6324 | Test Acc: 0.6557
K=8 | Seed 28 | Epoch 006 | Loss: 0.6138 | Test Acc: 0.6886
K=8 | Seed 28 | Epoch 007 | Loss: 0.6049 | Test Acc: 0.6436
K=8 | Seed 28 | Epoch 008 | Loss: 0.5949 | Test Acc: 0.6472
K=8 | Seed 28 | Epoch 009 | Loss: 0.5881 | Test Acc: 0.6363
K=8 | Seed 28 | Epoch 010 | Loss: 0.5821 | Test Acc: 0.6618
K=8 | Seed 28 | Epoch 011 | Loss: 0.5789 | Test Acc: 0.7044
K=8 | Seed 28 | Epoch 012 | Loss: 0.5819 | Test Acc: 0.6752
K=8 | Seed 28 | Epoch 013 | Loss: 0.5685 | Test Acc: 0.7019
K=8 | Seed 28 | Epoch 014 | Loss: 0.5680 | Test Acc: 0.6959
K=8 | Seed 28 | Epoch 015 | Loss: 0.5638 | Test Acc: 0.6703
K=8 | Seed 28 | Epoch 016 | Loss: 0.5564 | Test Acc: 0.6837
K=8 | Seed 28 | Epoch 017 | Loss: 0.5543

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁
test_acc,▄▁▃▅▅▅▅▅▅▇▇▇▆▆▇█▇▄██▅▅▇▇▇██▆▇███▇████▇██
train_acc,▁▁▃▄▅▅▅▅▅▆▆▆▆▆▆▆▇▆▇▆▆▆▆▆▇▇▇▇▇███▇███████
epoch,50
loss,0.4898
test_acc,0.73236
train_acc,0.76217


K=8 | Seed 29 | Epoch 001 | Loss: 0.6888 | Test Acc: 0.6010
K=8 | Seed 29 | Epoch 002 | Loss: 0.6747 | Test Acc: 0.6460
K=8 | Seed 29 | Epoch 003 | Loss: 0.6640 | Test Acc: 0.6460
K=8 | Seed 29 | Epoch 004 | Loss: 0.6494 | Test Acc: 0.6630
K=8 | Seed 29 | Epoch 005 | Loss: 0.6326 | Test Acc: 0.6800
K=8 | Seed 29 | Epoch 006 | Loss: 0.6272 | Test Acc: 0.6350
K=8 | Seed 29 | Epoch 007 | Loss: 0.6168 | Test Acc: 0.6715
K=8 | Seed 29 | Epoch 008 | Loss: 0.6056 | Test Acc: 0.6667
K=8 | Seed 29 | Epoch 009 | Loss: 0.5989 | Test Acc: 0.6788
K=8 | Seed 29 | Epoch 010 | Loss: 0.5946 | Test Acc: 0.6630
K=8 | Seed 29 | Epoch 011 | Loss: 0.5884 | Test Acc: 0.6886
K=8 | Seed 29 | Epoch 012 | Loss: 0.5849 | Test Acc: 0.6813
K=8 | Seed 29 | Epoch 013 | Loss: 0.5839 | Test Acc: 0.6752
K=8 | Seed 29 | Epoch 014 | Loss: 0.5763 | Test Acc: 0.6910
K=8 | Seed 29 | Epoch 015 | Loss: 0.5708 | Test Acc: 0.6971
K=8 | Seed 29 | Epoch 016 | Loss: 0.5711 | Test Acc: 0.6983
K=8 | Seed 29 | Epoch 017 | Loss: 0.5634

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁
test_acc,▁▃▄▅▃▅▅▄▅▅▆▆▆▇▆▇▇▇▆▆▆▇▇▇▄▆▇▆▇█▆█▆▇▆▇▇▇▇█
train_acc,▁▂▃▃▄▄▄▅▄▅▅▅▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▆▇▆▆▆█▇██▇██
epoch,50
loss,0.5039
test_acc,0.72141
train_acc,0.76734
